# OpenAI - Test API Generation

In this notebook, we create a fresh REST API specification. APIGeneration models are thought to generate a new API based on similar APIs.

APIGenerator and APIFormatter and thought to perform together but are not exclusive.


## 1. Data Preparation - API Examples and instructions.

The examples in the `api_examples` folder are APIs along with their descriptions, enabling a search based on similarity.

The variable `instructions` pertains to specific criteria we want the generated API to meet. This could involve specifying that it should be entirely in English or include certain types of responses.

In [1]:
import sys; sys.path.append('..')

import os
import json
from promptmeteo import APIGenerator
from promptmeteo import APIFormatter

with open("../data/api_examples/api_rest.json") as fin:
    data = json.load(fin)
    api_descriptions = [i["description"] for i in data]
    api_codes = [i["api"] for i in data]

instructions = [
    {
        "rule": "Actions `get`, `post`, `update`, `delete` must have the properties `tags`, `description`, `summary`, `operationId`, `parameters`, `response`.",
        "example": "```\n    paths:\n      /products:\nget:\n  tags:\n  - Products\n  summary: Service to fetch the list of products.\n  description: This service fetches the list of the products.\n  operationId: findAllProducts\n  parameters:\n  - name: brand\n  responses:\n    '200':\n      description: OK\n    ```"
    }
]

The following variables are essential in the inference process:

- **description:** This is the phrase that provides meaning to the API.
- **protocol and version:** These variables are related to the technical specification of the generated code.
- **additional information:** This variable allows us to include custom information. This information is incorporated exactly as specified in the variable.

In [2]:
description="An API for managing my finances"
protocol="REST"
version="3.0.3"
additional_info={
    "servers": [
        {
            "url": "http://localhost:8080/",
            "description": "Local environment",
        }
    ],
}

## 2. APIGenerator - Generate a new API.

API Generator is in charge of creating a new REST API based on a knowledge base and some instructions. Example APIs are search based on their descriptions matching the description of the new API.

In [3]:
model_1 = APIGenerator(
        language="en",
        n_samples=5,
        api_version=version,
        api_protocol=protocol,
        api_style_instructions=[i["rule"] for i in instructions],
        model_provider_name="openai",
        model_name="gpt-3.5-turbo-16k",
        model_provider_token=os.environ.get(
            "OPENAI_API_KEY",
            "",
        ),
        verbose=True,
    ).train(api_descriptions, api_codes)

In [4]:
api = model_1.predict([description])[0]



PROMPT INPUT

 You are an API Designer that has to create an API file for your company. The expected file should be OpenAPI 3.0.3 REST API YAML. 

### RULES ###

 The API should fulfill the following rules: 
 - Actions `get`, `post`, `update`, `delete` must have the properties `tags`, `description`, `summary`, `operationId`, `parameters`, `response`. 

### EXAMPLES ###

For doing so take into consideration `paths`, `schemas` and `parameters` from the following API (They are not complete, they just include the scaffolding):

IBANAPI OpenApi documentation
```
openapi: 3.0.0
servers:
- description: IBANAPI Server
  url: https://api.ibanapi.com/v1
info:
  contact:
    email: samy@ibanapi.com
  description: IBANAPI OpenApi documentation
  title: IBANAPI OpenApi Documentation
  version: 1.0.0
  x-apisguru-categories:
  - financial
  x-logo:
    url: https://api.apis.guru/v2/cache/logo/https_ibanapi.com_assets_favicon_apple-touch-icon.png
  x-origin:
  - format: openapi
    url: https://api

## API Formatter - Correct an API

The API Formatter processes an API specification, providing instructions to the Language Model (LLM) for correction and formatting based on the given instructions. Additionally, its parser handles schemas and parameters, replacing coincidental information with details extracted from example APIs. The objective is to maximize the reuse of information throughout the formatting process.

In [5]:
model_2 = APIFormatter(
        language="en",
        api_version=version,
        api_protocol=protocol,
        api_style_instructions=[
            "For example:\n\n".join(i.values()) for i in instructions
        ],
        model_provider_name="openai",
        model_name="gpt-3.5-turbo-16k",
        model_provider_token=os.environ.get(
            "OPENAI_API_KEY",
            "",
        ),
        verbose=True,
    ).train(api_codes)

In [7]:
api = model_2.predict([api], additional_info)[0]



PROMPT INPUT

 I want you to complete and correct an openAPI file. The expected file should be OpenAPI 3.0.3 REST API YAML. 

### RULES ###


Make sure that the API should fulfill the following requirements: 
 - Actions `get`, `post`, `update`, `delete` must have the properties `tags`, `description`, `summary`, `operationId`, `parameters`, `response`.For example:

```
    paths:
      /products:
get:
  tags:
  - Products
  summary: Service to fetch the list of products.
  description: This service fetches the list of the products.
  operationId: findAllProducts
  parameters:
  - name: brand
  responses:
    '200':
      description: OK
    ```
The API that you have to correct is:
```
openapi: 3.0.3
servers:
- url: http://localhost:8080/
  description: Local environment
info:
  contact:
    email: api@myfinances.com
  description: An API for managing my finances.
  title: My Finances API
  version: 1.0.0
tags:
- description: API Endpoints of My Finances
  name: My Finances
paths:
  /b

AttributeError: 'str' object has no attribute 'get'

In [8]:
print(api)

openapi: 3.0.3
servers:
- url: http://localhost:8080/
  description: Local environment
info:
  contact:
    email: api@myfinances.com
  description: An API for managing my finances.
  title: My Finances API
  version: 1.0.0
tags:
- description: API Endpoints of My Finances
  name: My Finances
paths:
  /balance:
    get:
      tags:
      - My Finances
      summary: Get Balance
      description: Get the balance of my finances.
      operationId: getBalance
      parameters:
      - $ref: '#/components/parameters/accessToken'
      responses:
        '200':
          description: Successful response with the balance.
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/BalanceResponse'
        '401':
          $ref: '#/components/schemas/401'
  /transactions:
    get:
      tags:
      - My Finances
      summary: Get Transactions
      description: Get a list of transactions.
      operationId: getTransactions
      paramet

&nbsp;


&nbsp;
